In [56]:
import numpy as np
import scipy.stats as sps
from math import exp

from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool

class InputNeuron(object):
    
    def __init__(self, nnet, spike_train):
        self.spike_train = spike_train
        self.output_spikes_times = []
        self.net = nnet
        
    def set_spike_train(self, spike_train):
        self.spike_train = spike_train
        self.output_spikes_times = []
        
    def step(self):
        if self.spike_train[self.net.global_time] == 1:
            self.output_spikes_times.append(self.net.global_time)
    
    def get_spikes(self):
        return self.output_spikes_times

class Neuron(object):
    
    def __init__(self, nnet):
        self.input_spikes = [] # pairs time, intensity
        self.output_spikes_times = []
        self.net = nnet
        self.potential = 0
        self.threshold = 0.01
        self.tau_m = 4
        self.tau_s = 2
        self.tau_r = 20
        self.time_scale = 1. # time unit is 100 ms
        self.history = [0]
        self.last_output_spikes_time = 0
    
    #очистить все после предыдущего запуска сети
    def restart(self):
        self.input_spikes = [] # pairs time, intensity
        self.output_spikes_times = []
        self.history = [0]
    
    def receive_spike(self, intensity):
        self.input_spikes.append((self.net.global_time, intensity))
    
    def step(self):
        self.potential = 0
        self.spike = False

        global_time = self.net.global_time
        time_scale = self.time_scale
        potential = 0
        for spike_time, intensity in self.input_spikes:
            if time_scale * (global_time - spike_time) < 30 and (intensity < -0.0001 or intensity > 0.0001):
                potential += self.eps(global_time - spike_time) * intensity
        self.potential = potential

        self.potential += self.nu(self.net.global_time - self.last_output_spikes_time)

        if self.potential > self.threshold:
            self.input_spikes = []
            self.output_spikes_times.append(self.net.global_time)
            self.last_output_spikes_time = self.net.global_time
            self.spike = True
            # print("Spike!")

        self.history.append(self.potential)

    def eps(self, time):
        if time <= 0:
            return 0
        s = -abs(time * self.time_scale)
        return exp(s / self.tau_m) - exp(s / self.tau_s)

    def grad_eps(self, time):
        if time <= 0:
            return 0
        s = -abs(time * self.time_scale)
        return exp(s / self.tau_m) * (-1 / self.tau_m) - exp(s / self.tau_s) * (-1 / self.tau_s)

    def nu(self, time):
        if time <= 0:
            return 0
        s = time * self.time_scale
        return -self.threshold * exp(-abs(s) / self.tau_r) * (s > 0)

    def get_spikes(self):
        return self.output_spikes_times


class Connection(object):
    def __init__(self, nnet, input_neuron, output_neuron, weights=[1], delays=[1],
                etta=4, tau_min=6, tau_plus=3, sigma=0.1):  # weights and delays are scaled
        self.weights = weights
        self.delays = delays
        self.input_neuron = input_neuron
        self.output_neuron = output_neuron
        self.net = nnet
        self.last_conducted_spike_index = 0
        
        #for STDP
        self.etta = etta
        self.tau_min = tau_min
        self.tau_plus = tau_plus
        self.sigma = sigma

    def step(self):
        spikes = self.input_neuron.output_spikes_times
        for i in range(self.last_conducted_spike_index, len(spikes)):
            spike_time = spikes[i]
            for j in range(len(self.weights)):
                if spike_time + self.delays[j] == self.net.global_time:
                    self.last_conducted_spike_index += 1
                    self.output_neuron.receive_spike(self.weights[j])

    def gradient(self, expected_spike_time):
        output_spikes = np.array(self.output_neuron.get_spikes())
        input_spikes = np.array(self.input_neuron.get_spikes())
        assert len(output_spikes) > 0
        output_spike_time = output_spikes[0]
        eps = self.output_neuron.eps
        grad_eps = self.output_neuron.grad_eps
        grad_numerator = 0
        grad_denumerator = 0
        for i in range(len(input_spikes)):
            grad_numerator += eps(output_spike_time - input_spikes[i] - self.delays)
            # sum over all delays and input axons for output_neuron
            grad_denumerator += self.weights * grad_eps(output_spike_time - input_spikes[i] - self.delays)
        # leaning rate will be in a learning algorithm
        grad = -grad_numerator / np.maximum(0.1, grad_denumerator) * (output_spike_time - expected_spike_time)
        return grad
    
    ############################
    def STDP(self):
        output_spikes = np.array(self.output_neuron.get_spikes())
        input_spikes = np.array(self.input_neuron.get_spikes())
        weights = np.array(self.weights)
        w_max = weights.max()
        w_min = weights.min()
        out_spikes_num = len(output_spikes)
        
        if out_spikes_num <= 0:
            return
        
        tau_post_before = 0
        tau_post_after = output_spikes[0]
        out_ind = 0
        
        etta = self.etta
        tau_min = self.tau_min
        tau_plus = self.tau_plus
        sigma = self.sigma
        
        # для каждого синапса нужно поменять его вес
        for j in range(len(weights)):
            for i, tau_pre in enumerate(input_spikes):
                #####################
                if (tau_pre > tau_post_after):
                    tau_post_before = tau_post_after
                    if (out_ind < out_spikes_num - 1):
                        out_ind += 1
                        tau_post_after = output_spikes[out_ind]
                    else:
                        tau_post_after = 0
                
                if (0 <= tau_pre-tau_post_after <=tau_min):
                    delta_w = -1.0*etta*(tau_min-(tau_pre-tau_post_after))
                elif (-1.*tau_plus <= tau_pre-tau_post_after <= 0):
                    delta_w = etta*(tau_plus+(tau_pre-tau_post_after))
                else:
                    delta_w = 0
                    
                if (0 <= tau_pre-tau_post_before <=tau_min):
                    delta_w -= etta*(tau_min-(tau_pre-tau_post_before))
                elif (-1.*tau_plus <= tau_pre-tau_post_before <= 0):
                    delta_w += etta*(tau_plus+(tau_pre-tau_post_before))

                w_old = self.weights[j]
                if (delta_w > 0):
                    self.weights[j] = w_old + sigma*delta_w*(w_max - w_old)
                else:
                    self.weights[j] = w_old + sigma*delta_w*(w_old - w_min)

In [53]:
net = NNet((1,1,1))
in_n = Neuron(net)
out_n = Neuron(net)
#out_n.
con = Connection(net, in_n, out_n)
#con.STDP()

In [36]:
def fixed_frequency_spike_train(frequency, t_max):
    actual_frequency = float(frequency)
    result = [1 if frequency > 0 else 0]
    for i in range(t_max - 1):
        if actual_frequency >= 1000:
            result.append(1)
            actual_frequency -= int(actual_frequency / 1000) * 1000
        else:
            result.append(0)
        actual_frequency += frequency
    return result

from functools import reduce
class InputLayer(object):
    def __init__(self, nnet, shape):
        self.net = nnet
        self.shape = shape 
        self.neur_size = reduce(lambda res, x: res*x, self.shape, 1)
        self.neurons = np.ndarray(shape=self.shape, dtype=InputNeuron, buffer=np.array([InputNeuron(self.net, []) for i in np.arange(self.neur_size)]))
            
    def random_spike_train(freq, t_max):
        return sps.bernoulli.rvs(freq/4.+ 1./4, size=t_max)
    
    def new_input(self, arg, make_spike_train=random_spike_train, t_max=1000):
        for i, f in enumerate(arg):
            for j, l in enumerate(f):
                for k, m in enumerate(l):
                    self.neurons[i][j][k].set_spike_train(make_spike_train(arg[i][j][k], t_max))
    
    def step(self):
        for neur in self.neurons.reshape((self.neur_size)):
            neur.step()
            
    def learning(self):
        pass

In [37]:
class Conv2DLayer(object):
    # Формат весов: w[nk][h][i][j], где nk - фильтр, h - номер фильтра на предыдущем слое, i, j - координаты весов в фильтре
    def __init__(self, nnet, input_layer, num_filters, filter_shape, weights):
        self.net = nnet
        self.filter_shape = filter_shape
        self.weights = self.oldweights = weights.copy() 
        if(len(self.weights.shape) < 5):
            self.weights = self.weights.reshape(np.append(weights.shape, 1))
        
        self.shape = (num_filters, input_layer.shape[1]-filter_shape[0]+1, input_layer.shape[2]-filter_shape[1]+1)
        self.neur_size = reduce(lambda res, x: res*x, self.shape, 1)
        self.neurons = np.array([Neuron(self.net) for i in np.arange(self.neur_size)]).reshape(self.shape)
        
        self.connections = []
        
        for nk, kernel in enumerate(self.neurons):
            for i, row in enumerate(kernel):
                for j, neuron in enumerate(row):   # соединяем с предыдущим слоем
                    self.connections += [Connection(self.net, input_layer.neurons[l][i+p][j+q],neuron, self.weights[nk][l][p][q])\
                                         for l in np.arange(input_layer.shape[0]) for p in np.arange(filter_shape[0])\
                                         for q in np.arange(filter_shape[1])] 
                    
    def restart(self):
        for i, neur in enumerate(self.neurons.reshape((self.neur_size))):
            neur.restart()
    
    def step(self):
        for conn in self.connections:
            conn.step()
        for i, neur in enumerate(self.neurons.reshape((self.neur_size))):
            neur.step()
            
    def learning(self):
        for conn in self.connections:
            conn.STDP()

In [38]:
class SubSampling2DLayer(object):
    def __init__(self, nnet, input_layer, pool_size):
        self.net = nnet
        self.pool_size = pool_size
        self.shape = input_layer.shape // np.append([1], pool_size)
        self.neur_size = reduce(lambda res, x: res*x, self.shape, 1)
        self.neurons = np.array([Neuron(self.net) for i in np.arange(self.neur_size)]).reshape(self.shape)
        
        self.conn_weight = 1 / (pool_size[0] * pool_size[1])
        
        self.connections = []
        
        for nk, kernel in enumerate(self.neurons):
            for i, row in enumerate(kernel):
                for j, neuron in enumerate(row):   
                    self.connections += [Connection(self.net,input_layer.neurons[l][i*pool_size[0]+p][j*pool_size[1]+q],neuron,\
                                                    [self.conn_weight])\
                                         for l in np.arange(input_layer.shape[0]) for p in np.arange(pool_size[0])\
                                         for q in np.arange(pool_size[1])] 
                    
    def restart(self):
        for i, neur in enumerate(self.neurons.reshape((self.neur_size))):
            neur.restart()
        
    def step(self):
        for conn in self.connections:
            conn.step()
        for neur in self.neurons.reshape((self.neur_size)):
            neur.step()
            
    def learning(self):
        pass

In [39]:
class DenseLayer(object):
    #Формат весов: w[i][j],  где i - номер нейрона на предыдущем слое, j - номер нейрона на текущем слое
    def __init__(self,nnet, input_layer, num_units, weights, workers=10):
        self.net = nnet
        self.shape = [num_units]
        self.neur_size = num_units
        self.neurons = np.array([Neuron(self.net) for i in np.arange(self.neur_size)])
        self.pool = ThreadPool(workers)
        self.weights = weights
        
        if(len(weights.shape) < 3):
            weights = weights.reshape(np.append(weights.shape, 1))
        
        self.connections = [Connection(self.net, input_neuron, output_neuron, weights[i][j])\
                            for i, input_neuron in enumerate(input_layer.neurons.reshape((input_layer.neur_size)))\
                            for j, output_neuron in enumerate(self.neurons)]
        
    def restart(self):
        for neur in self.neurons:
            neur.restart()
        
    def step(self):
        #print("NewLayer!")
        #for conn in self.connections:
            #conn.step()
        self.pool.map(lambda x: x.step(), self.connections)
        self.pool.map(lambda x: x.step(), self.neurons)
        #for neur in self.neurons:
            #neur.step()
            
    def learning(self):
        for conn in self.connections:
            conn.STDP()

In [40]:
class NNet(object):
    def __init__(self, shape, workers=5):
        self.layers = [InputLayer(self, shape)]
        self.global_time = 0
        self.pool = ThreadPool(workers)
    
    def add_convolution(self, weights):
        num_filters = weights.shape[0]
        filter_shape = weights.shape[2:4]
        self.layers.append(Conv2DLayer(self, self.layers[-1], num_filters, filter_shape, weights))
        
    def add_subsampling(self, pool_size):
        self.layers.append(SubSampling2DLayer(self, self.layers[-1], pool_size))
        
    def add_dense(self, weights):
        num_units = weights.shape[1]
        self.layers.append(DenseLayer(self, self.layers[-1], num_units, weights))
    
    def get_output_for(self, data, t_max):
        self.global_time = 0
        self.layers[0].new_input(data)
        for l in self.layers[1:]:
            l.restart()
        for t in np.arange(t_max):
            for layer in self.layers:
                layer.step()
            self.global_time += 1
        result = [neur.get_spikes() for neur in self.layers[-1].neurons.reshape((self.layers[-1].neur_size))]
        return result
    
    def classify(self, data, t_max):
        self.global_time = 0
        self.layers[0].new_input(data)
        for l in self.layers[1:]:
            l.restart()
        ans = []
        for t in np.arange(t_max):
            #for layer in self.layers:
                #layer.step()
            self.pool.map(lambda x: x.step(), self.layers)
            for i, neur in enumerate(self.layers[-1].neurons):
                if len(neur.get_spikes()) > 0:
                    ans.append(i)
            if(len(ans) > 0):
                return ans, t
            self.global_time += 1
        print('not_enough_time')
        
    def learning(self):
        for layer in self.layers:
            layer.learning()

In [41]:
import lasagne

def spiking_from_lasagne(input_net):
    input_layers = lasagne.layers.get_all_layers(input_net)
    weights = lasagne.layers.get_all_param_values(input_net)
    spiking_net = NNet(input_layers[0].shape[-3:])
    convert_layers = {lasagne.layers.conv.Conv2DLayer : spiking_net.add_convolution,\
                      lasagne.layers.dense.DenseLayer : spiking_net.add_dense}
    
    #номер элемента в общем массиве весов, в котором хранятся веса текущего слоя
    i = 0
    
    for l in input_layers[1:]:
        if(type(l) == lasagne.layers.pool.Pool2DLayer or type(l) == lasagne.layers.pool.MaxPool2DLayer):
            spiking_net.add_subsampling(l.pool_size)
        else:
            convert_layers[type(l)](weights[i])
            i+=1
            
    return spiking_net